# Introduction:

Weather prediction is a crucial aspect of daily life, affecting everything from agriculture to transportation to emergency preparedness. While traditional forecasting methods have come a long way, there is still much room for improvement in accuracy and efficiency. In recent years, machine learning (ML) has emerged as a promising approach to weather prediction. By leveraging the vast amounts of data collected from weather stations around the world, ML models can analyze complex patterns and relationships to make more accurate and timely predictions.

In this research paper, we explore the application of ML to weather prediction. Specifically, we focus on the use of supervised learning algorithms, including decision trees, logistic regression, and k-nearest neighbors, to predict weather conditions based on historical data. We use a dataset containing daily weather measurements from multiple weather stations in a particular region and train our ML models on this data to predict future weather conditions.

Our research aims to address several key questions, including: What types of ML algorithms are best suited for weather prediction? How does the size and quality of the training dataset affect the accuracy of the predictions? What features or variables have the most significant impact on weather prediction accuracy? By answering these questions, we hope to shed light on the potential of ML for weather prediction and provide insights into best practices for using these technologies in real-world applications.

Overall, this research has the potential to make a significant contribution to the field of weather prediction and inform future research in this area. By demonstrating the effectiveness of ML for predicting weather conditions, we hope to inspire further exploration and innovation in this important domain.

*   In this report, we will use the "seattle-weather.csv" dataset from Kaggle to analyze and build predictive models of weather conditions based on accompanying conditions. This is a set of weather data in the US state of Seattle recorded on a daily basis for about 4 years, includes weather conditions such as observed date, high and low temperature, precipitation, speed wind and weather conditions of the day associated with those conditions. [Link dataset from Kaggle](https://www.kaggle.com/datasets/ananthr1/weather-prediction)

*   Models used: Logistic Regression, Decision Tree, K-Nearest Neighbor Classifier (KNN).

# **Import Library**

imports necessary libraries such as numpy, pandas, seaborn, etc. for data manipulation, visualization, statistical analysis, and machine learning.

In [1]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import scipy
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, ttest_ind
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# **Retrieve the Dataset**

Use the open meteo API to get the data: https://open-meteo.com/en/docs#hourly=temperature_2m,relative_humidity_2m,rain,snowfall 

In [2]:

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 48.2085,
	"longitude": 16.3721,
	"start_date": "2010-06-10",
	"end_date": "2024-06-24",
	"hourly": ["temperature_2m", "relative_humidity_2m", "precipitation", "rain", "snowfall", "cloud_cover", "wind_speed_10m", "wind_direction_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(2).ValuesAsNumpy()
hourly_rain = hourly.Variables(3).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(4).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(5).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(6).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(7).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

  Using cached retry_requests-2.0.0-py3-none-any.whl.metadata (2.6 kB)
Using cached retry_requests-2.0.0-py3-none-any.whl (15 kB)



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\patri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached openmeteo_requests-1.2.0-py3-none-any.whl.metadata (7.6 kB)
  Using cached openmeteo_sdk-1.11.8-py3-none-any.whl.metadata (13 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
Using cached openmeteo_requests-1.2.0-py3-none-any.whl (5.5 kB)
Using cached openmeteo_sdk-1.11.8-py3-none-any.whl (12 kB)
Using cached flatbuffers-24.3.25-py2.py3-none-any.whl (26 kB)



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\patri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: C:\Users\patri\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached cattrs-23.2.3-py3-none-any.whl.metadata (10 kB)
  Using cached url_normalize-1.4.3-py2.py3-none-any.whl.metadata (3.1 kB)
Using cached requests_cache-1.2.1-py3-none-any.whl (61 kB)
Using cached cattrs-23.2.3-py3-none-any.whl (57 kB)
Using cached url_normalize-1.4.3-py2.py3-none-any.whl (6.8 kB)
Coordinates 48.18980407714844°N 16.377296447753906°E
Elevation 196.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                            date  temperature_2m  relative_humidity_2m  \
0      2010-06-10 00:00:00+00:00       20.069500             72.250412   
1      2010-06-10 01:00:00+00:00       19.419500             73.307968   
2      2010-06-10 02:00:00+00:00       18.869501             73.687531   
3      2010-06-10 03:00:00+00:00       18.369501             74.563713   
4      2010-06-10 04:00:00+00:00       18.119501             76.981262   
...                          ...             ..